In [1]:
#A model that takes human images and then classisifes them according to the emotion detected
# detects only happy, sad and anger

In [19]:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import keras
from keras.utils import to_categorical
import os
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.preprocessing.image import img_to_array
from keras.applications.imagenet_utils import decode_predictions
import matplotlib.pyplot as plt
%matplotlib inline
from keras.applications import vgg16
from keras.models import Sequential, load_model
from keras.callbacks import ModelCheckpoint

In [20]:
from keras.applications import VGG16

vgg_conv = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3))

In [21]:
print(vgg_conv.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [36]:
train_dir = 'images/train_3_additional'
evaluation_dir='images/evaluation_3_emotion'
# there are 6 folders (classes) of images
# each has 650 usable photos
nTrain = 480 #620 x 3
nEval= 180 #60 x 3
#The next step takes a lot of time depending on your machine

In [37]:
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

train_features = np.zeros(shape=(nTrain, 7, 7, 512))
train_labels = np.zeros(shape=(nTrain,3))
#I HAVE 3 classes now

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)

i = 0
for inputs_batch, labels_batch in train_generator:
    features_batch = vgg_conv.predict(inputs_batch)
    train_features[i * batch_size : (i + 1) * batch_size] = features_batch
    train_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= nTrain:
        break
        
train_features = np.reshape(train_features, (nTrain, 7 * 7 * 512))

Found 512 images belonging to 3 classes.


In [24]:
evaluation_features = np.zeros(shape=(nEval, 7, 7, 512))
evaluation_labels = np.zeros(shape=(nEval,3))
# 3 here as well

evaluation_generator = datagen.flow_from_directory(
    evaluation_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

i = 0
for inputs_batch, labels_batch in evaluation_generator:
    features_batch = vgg_conv.predict(inputs_batch)
    evaluation_features[i * batch_size : (i + 1) * batch_size] = features_batch
    evaluation_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= nEval:
        break

evaluation_features = np.reshape(evaluation_features, (nEval, 7 * 7 * 512))

Found 228 images belonging to 3 classes.


In [25]:
save_path='images/'

In [30]:
from keras import models
from keras import layers
from keras import optimizers

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(512, activation='relu', input_dim=7 * 7 * 512))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(3, activation='softmax'))

    model.compile(optimizer=optimizers.RMSprop(lr=2e-4),
                  loss='categorical_crossentropy',
                  metrics=['acc'])
    print(model.summary())
    return model
def train_model(model,reload_filename='NULL',save_best_only=True,batch_size=20,num_epochs=10):
    if(save_best_only):
        checkpointer = ModelCheckpoint(filepath=save_path + '/model-{epoch:02d}-{loss:.4f}.hdf5', verbose=1,monitor='val_acc', save_best_only=True, mode='auto')
    else:
        checkpointer = ModelCheckpoint(filepath=save_path + '/model-{epoch:02d}-{loss:.4f}.hdf5', verbose=1)
    #period option in checkpointer -> after how many epochs to save the model
    if(reload_filename!="NULL"):
            model = load_model(save_path + reload_filename)
    model.fit(train_features,
                    train_labels,
                    epochs=num_epochs,
                    batch_size=20,
                    validation_data=(evaluation_features,evaluation_labels), callbacks=[checkpointer])
    #model.save(save_path + "final_model.hdf5")

In [31]:
model=build_model()
#reload_filename=".hdf5"

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 512)               12845568  
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 1539      
Total params: 12,847,107
Trainable params: 12,847,107
Non-trainable params: 0
_________________________________________________________________
None


In [38]:
reload_filename="model-07-0.4975.hdf5"

In [39]:
train_model(model,reload_filename=reload_filename)

Train on 480 samples, validate on 180 samples
Epoch 1/10
480/480 [==============================] - 7s 15ms/step - loss: 0.9268 - acc: 0.6083 - val_loss: 0.7483 - val_acc: 0.6111

Epoch 00001: val_acc improved from -inf to 0.61111, saving model to images//model-01-0.9268.hdf5
Epoch 2/10
480/480 [==============================] - 6s 13ms/step - loss: 0.5077 - acc: 0.7896 - val_loss: 0.5252 - val_acc: 0.8944

Epoch 00002: val_acc improved from 0.61111 to 0.89444, saving model to images//model-02-0.5077.hdf5
Epoch 3/10
480/480 [==============================] - 6s 13ms/step - loss: 0.3210 - acc: 0.8708 - val_loss: 0.6335 - val_acc: 0.7944

Epoch 00003: val_acc did not improve from 0.89444
Epoch 4/10
480/480 [==============================] - 6s 13ms/step - loss: 0.2954 - acc: 0.8688 - val_loss: 0.7371 - val_acc: 0.8944

Epoch 00004: val_acc did not improve from 0.89444
Epoch 5/10
480/480 [==============================] - 7s 14ms/step - loss: 0.2169 - acc: 0.9104 - val_loss: 0.8563 - val_

In [40]:
def test_model_accuracy(filename):
    model = load_model("images/" + filename)
    scores = model.evaluate(evaluation_features,evaluation_labels, verbose=1)
    print("Accuracy: %.2f%%" % (scores[1]*100))
filename="model-07-0.1495.hdf5"
#change according to the file generated
test_model_accuracy(filename)

180/180 [==============================] - 0s 2ms/step
Accuracy: 90.00%


In [22]:
#expand dataset; Try training a fresh model; Then test accuracy; Then try alternatives to VGG
#Stick to 3 emotions
#now expand dataset
#problem is that the different datasets are of different sizes, so how do you bring them all to the same size
#also how do you bring my pictures to the front

#now new option do you want to only train with the images you took ?
#yup